In [1]:
import sys, numpy as np
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

images = x_train[0:1000].reshape(1000,28*28) / 255 
 
labels = y_train[0:1000] #итого с чем сверяться.

one_hot_labels = np.zeros((len(labels),10)) #зануляем выходные результаты. размерностью 10.
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels        # оцифровываем 3 в 001.., 2 в 010... и т д ???


test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

In [ ]:
import numpy as np
import sys
np.random.seed(1)
def relu(x):
    return (x>=0) * x
    
def relu2deriv(output):
    return output >= 0
    
batch_size = 100
alpha, iterations = (0.001 , 1000)
pixels_per_image, num_labels, hidden_size = (784, 10, 100)

#задаем произвольно веса

weights_0_1 = 0.2*np.random.random((pixels_per_image, hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1

# главный цикл итерации

for j in range(iterations):
    error, correct_cnt = (0.0 , 0)
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i+1)*batch_size))
        
        layer_0 = images[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = np.dot(layer_1, weights_1_2)
        
        error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2) 
        for k in range(batch_size):
                correct_cnt += int(np.argmax(layer_2[k:k+1]) == \
                        np.argmax(labels[batch_start+k:batch_start+k+1]))
                layer_2_delta = (labels[batch_start:batch_end]-layer_2) \
                                                        /batch_size
                layer_1_delta = layer_2_delta.dot(weights_1_2.T)* \
                                                    relu2deriv(layer_1)
                layer_1_delta *= dropout_mask
                
                weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
                weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
                
    if (j%10 ==0):
            test_error = 0.0
            test_correct_cnt = 0
                
            for i in range(len(test_images)):
                layer_0 = test_images[i:i+1]
                layer_1 = relu(np.dot(layer_0,weights_0_1))
                layer_2 = np.dot(layer_1, weights_1_2)
                
                test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
                test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
                
            sys.stdout.write("\n" + \
                "I:" + str(j) + \
                " Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
                " Test-Acc:" + str(test_correct_cnt / float(len(test_images))) +\
                " Train-Ecc:" + str(error / float(len(images)))[0:5] +\
                " Train-Acc:" + str(correct_cnt/ float(len(images))))

In [2]:
import numpy as np
import sys
np.random.seed(1)
def relu(x):
    return (x>=0) * x
    
def relu2deriv(output):
    return output >= 0
    
batch_size = 100
alpha, iterations = (0.001 , 20)

#Размеры. Входящий слой, выходной слой, промежуточный слой.
pixels_per_image, num_labels, hidden_size = (784, 10, 100)

#Задаем произвольно веса
#Создаем матрицу весов между 0-ым и 1-ым слоем. 
weights_0_1 = 0.2*np.random.random((pixels_per_image, hidden_size)) - 0.1
#Создаем матрицу весов между 1-ым и 2-ым слоем. 
weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1

#Главный цикл итерации

for j in range(iterations):
    error, correct_cnt = (0.0 , 0)      #Инициализируем ошибки
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i+1)*batch_size))
        
        layer_0 = images[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0, weights_0_1)) #вектор
        dropout_mask = np.random.randint(2,size=layer_1.shape) #выключаем часть слоя 1 через его размерность. 
        layer_1 *= dropout_mask * 2 #будут активированы не всё в слое только 50%. почему умножают на 2 - выключают половину, 
                                    #поэтому надо будет суммы увеличить в двое, чтобы следующий слой не чувстоввал подвоха.
        layer_2  = np.dot(layer_1, weights_1_2)
        
        error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2) #считаем ошибку
        for k in range(batch_size):
            #обратный ход
                #определение прогноза с фактом. счетчик.
                correct_cnt   += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))
                layer_2_delta =  (labels[batch_start:batch_end]-layer_2)  / batch_size
                layer_1_delta =  layer_2_delta.dot(weights_1_2.T)*  relu2deriv(layer_1)
                #Только для тех, что выключали
                layer_1_delta *= dropout_mask
                #корректируем веса
                weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
                weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

                #на шаге кратный 10 проверяем результат. тестовая выборка.
    if (j%10 ==0):
            test_error = 0.0
            test_correct_cnt = 0
                
            for i in range(len(test_images)):
                layer_0 = test_images[i:i+1]
                layer_1 = relu(np.dot(layer_0,weights_0_1))
                layer_2 = np.dot(layer_1, weights_1_2)
                
                test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
                test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

                #выводим статистику по результатам расчета
            sys.stdout.write("\n" + \
                "I:" + str(j) + \
                " Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
                " Test-Acc:" + str(test_correct_cnt / float(len(test_images))) +\
                " Train-Ecc:" + str(error / float(len(images)))[0:5] +\
                " Train-Acc:" + str(correct_cnt/ float(len(images))))


I:0 Test-Err:0.815 Test-Acc:0.3832 Train-Ecc:1.284 Train-Acc:0.165
I:10 Test-Err:0.568 Test-Acc:0.7173 Train-Ecc:0.591 Train-Acc:0.672

In [ ]:
print('weights_1_2=', weights_1_2)

In [ ]:
print('weights_0_1=', weights_0_1)

In [ ]:
len(weights_0_1)

In [ ]:
len(weights_1_2)

In [ ]:
w_0_1 = 0.6*np.random.random((7, 2)) - 0.1
w_0_1

In [ ]:
_layer_1 = np.random.randint(2,size=10)
_dropout_mask = np.random.randint(2,size=10)
_dropout_mask

In [ ]:
_layer_1
_layer_1 *= _dropout_mask * 2
_layer_1

In [ ]:
import sys, numpy as np
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

images = x_train[0:1000].reshape(1000,28*28) / 255 
 
labels = y_train[0:10] #итого с чем сверяться.

labels

In [ ]:
one_hot_labels = np.zeros((len(labels),10)) #зануляем выходные результаты. размерностью 10.
one_hot_labels

In [ ]:
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels
labels